# `tensorflow.keras` transfer learning from pretrained weights


This notebook demonstrates the use of (1) a well-known CNN architecture and (2) pretrained weights on `RadImageNet` in a multiclassification task.

References:

* The pretrained weights published by [Mei et al (2022)](https://pubs.rsna.org/doi/10.1148/ryai.210315) in their github [link](https://github.com/BMEII-AI/RadImageNet).

* The github repository form the BAGLS team contained in this github [link](https://github.com/anki-xyz/bagls/blob/master/Utils/DataGenerator.py#L109)


In [1]:
# dev convenience
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import PATHS

import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["WANDB_SILENT"] = "True"
os.environ["WANDB_NOTEBOOK_NAME"] = "04-tf-pretrained-cat.ipynb"

PROJECT_NAME = 'bagls-sh-project'
RUN_NAME = 'pretrained_from_radimagenet'
METRICS_TABLE_NAME = 'metrics_table'
GRADCAM_LAYER_NAME = "conv5_block3_out"

In [3]:
import wandb
print("W&B: ", wandb.__version__)
wandb.login()

# # manage logs
# import logging

# logger = logging.getLogger("wandb")
# logger.setLevel(logging.ERROR)

# logging.getLogger('tensorflow').disabled = True

W&B:  0.13.5


True

In [4]:
# tf loader
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras

In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1465012877303074205
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 15693390111703385598
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1"
]


2022-11-29 14:03:08.156058: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 14:03:09.210239: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1


In [6]:
import config
configs = config.nb_configs

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# initialize data generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=configs["validation_split"],
    rescale=configs["rescale"],
    width_shift_range=configs["width_shift_range"],
    height_shift_range=configs["height_shift_range"],
    shear_range=configs["shear_range"],
    zoom_range=configs["zoom_range"],
    fill_mode=configs["fill_mode"],
    horizontal_flip=configs["horizontal_flip"],
    rotation_range=configs["rotation_range"],
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rescale=configs["rescale"],
)

In [8]:
train_dir = configs["train_dir"]
test_dir = configs["test_dir"]

batch_size = configs["batch_size"]
class_names = configs["class_names"]
interpol = configs["interpol"]
cmap = configs["cmap"]
label_mode = configs["label_mode"]
labels = configs["labels"]
image_size = configs["image_size"]


train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="training",
)

val_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="validation",
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    shuffle=False, # do not shuffle for later evaluation, alphanum sort
)

configs.update({"val_steps": val_dataset.samples // configs["batch_size"]})

Found 52393 images belonging to 2 classes.
Found 2757 images belonging to 2 classes.
Found 3300 images belonging to 2 classes.


In [9]:
dropout_rate = 0.2
def define_model(pretrained):
    conv_base = pretrained(
        include_top=False,
        weights=PATHS.resnet50_weights,
        input_shape=(*image_size, 3),
        pooling='avg',
    )
    print("Num trainable at load:", len(conv_base.trainable_weights)) 
    conv_base.trainable = False
    print("Num trainable:", len(conv_base.trainable_weights)) 
    
    x = conv_base.output
    
    # layers at this stage are arbitrary
    # can be subjected to hyperparam tuning
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.Dropout(configs["dropout_rate"])(x)
    outputs = keras.layers.Dense(units=2, activation="softmax")(x)
    model = keras.Model(inputs=conv_base.input, outputs=outputs)
    return model

In [10]:
pretrained = keras.applications.ResNet50
tf.keras.backend.clear_session()
model = define_model(pretrained)

2022-11-29 14:03:11.917915: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1


Num trainable at load: 212
Num trainable: 0


In [11]:
from tensorflow.keras import metrics
thresh = configs["thresh"]
metrics_dict = {
    "ACC":  metrics.CategoricalAccuracy(name="ACC"),
    "AUC-ROC": metrics.AUC(name='ROC', curve='ROC'),
    "AUC-PR": metrics.AUC(name='PR', curve='PR'),
    "TP": metrics.TruePositives(name="TP", thresholds=thresh),
    "TN": metrics.TrueNegatives(name="TN", thresholds=thresh),
    "FP": metrics.FalsePositives(name="FP", thresholds=thresh),
    "FN": metrics.FalseNegatives(name="FN", thresholds=thresh),
}

In [12]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses

# opt = optimizers.Adam(learning_rate=1e-06)
opt = optimizers.Adam()
met = list(metrics_dict.values())

model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer=opt,
    metrics=met,
)

In [13]:
# verify arch
# base predictions with untrained classif head
base_preds = model.predict(test_dataset)
base_preds

2022-11-29 14:03:15.506292: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-29 14:03:18.593649: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


array([[0.56711364, 0.43288636],
       [0.5589048 , 0.44109514],
       [0.6036478 , 0.39635214],
       ...,
       [0.5314221 , 0.46857795],
       [0.49138957, 0.5086105 ],
       [0.50592834, 0.49407163]], dtype=float32)

In [14]:
from interpretation import ValLog, GRADCamLogger

# initialize run
run = wandb.init(
    project=PROJECT_NAME, 
    name=RUN_NAME,
    config=configs, 
    job_type='train',
)

wandb_callback = wandb.keras.WandbCallback(
    monitor="val_ROC",
    mode="max",
    save_model=True,
    save_graph=True,
    compute_flops=True,
)

callbacks = [
    wandb_callback,
    ValLog(generator=val_dataset, num_log_batches=1),
    GRADCamLogger(generator=test_dataset, 
                  layer_name=GRADCAM_LAYER_NAME, num_log_batches=1)
    
]

In [15]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=configs["epochs"], 
    shuffle=True,
    callbacks=callbacks,
)
run.finish()

2022-11-29 14:03:56.254981: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-11-29 14:03:56.255348: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-11-29 14:03:56.257815: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1
2022-11-29 14:03:56.274094: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.023ms.
  function_optimizer: function_optimizer did nothing. time = 0.002ms.



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/10
410/410 [==============================] - 1031s 3s/step - loss: 0.5224 - ACC: 0.0000e+00 - ROC: 0.8154 - PR: 0.8154 - TP: 38610.0000 - TN: 38610.0000 - FP: 13783.0000 - FN: 13783.0000 - val_loss: 0.4330 - val_ACC: 1.8136e-04 - val_ROC: 0.8813 - val_PR: 0.8840 - val_TP: 2196.0000 - val_TN: 2196.0000 - val_FP: 561.0000 - val_FN: 561.0000


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2022-11-29 14:21:23.070085: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_140342-1af38hqf/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


...

/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 9/10
410/410 [==============================] - 990s 2s/step - loss: 0.1324 - ACC: 0.0083 - ROC: 0.9904 - PR: 0.9906 - TP: 49749.0000 - TN: 49749.0000 - FP: 2644.0000 - FN: 2644.0000 - val_loss: 0.1005 - val_ACC: 0.0085 - val_ROC: 0.9949 - val_PR: 0.9950 - val_TP: 2663.0000 - val_TN: 2663.0000 - val_FP: 94.0000 - val_FN: 94.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_140342-1af38hqf/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 10/10
410/410 [==============================] - 987s 2s/step - loss: 0.1201 - ACC: 0.0117 - ROC: 0.9919 - PR: 0.9921 - TP: 50061.0000 - TN: 50061.0000 - FP: 2332.0000 - FN: 2332.0000 - val_loss: 0.0987 - val_ACC: 0.0156 - val_ROC: 0.9954 - val_PR: 0.9955 - val_TP: 2670.0000 - val_TN: 2670.0000 - val_FP: 87.0000 - val_FN: 87.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_140342-1af38hqf/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [16]:
run = wandb.init(project=PROJECT_NAME, job_type="inference", name=RUN_NAME)
model_at = run.use_artifact("model-" + RUN_NAME + ":latest")
model_dir = model_at.download()
print("model: ", model_dir)
best_model = keras.models.load_model(model_dir)

wandb: ERROR Failed to sample metric: GPU is lost


model:  ./artifacts/model-pretrained_from_radimagenet:v8


In [17]:
# test metrics
import utils

metrics_results = best_model.evaluate(test_dataset)
metrics_results = dict(zip(["loss"] + list(metrics_dict.keys()), 
                           metrics_results))
tp, fp, tn, fn = (metrics_results["TP"], metrics_results["FP"], 
                  metrics_results["TN"], metrics_results["FN"])

add_metrics = {
    "SENSITIVITY": utils.get_sensitivity(tp, fp, tn, fn),
    "SPECIFICTY": utils.get_specificity(tp, fp, tn, fn),
    "PPV": utils.get_ppv(tp, fp, tn, fn),
    "NPV": utils.get_npv(tp, fp, tn, fn),
    "F1" : utils.get_fbeta(tp, fp, tn, fn, beta=1),
}
metrics_results.update(add_metrics)

print(f"Metrics: \n", metrics_results)

columns = list(metrics_results.keys())
metrics_table = wandb.Table(columns=columns)
metrics_table.add_data(*metrics_results.values())
wandb.run.log({METRICS_TABLE_NAME : metrics_table})

# add logging of confusion matrix image from matplotlib
           
# get preds
trained_preds = best_model.predict(test_dataset)
run.finish()

wandb: ERROR Failed to serialize metric: division by zero


26/26 [==============================] - 25s 875ms/step - loss: 0.6612 - ACC: 0.0879 - ROC: 0.8897 - PR: 0.8700 - TP: 2779.0000 - TN: 2779.0000 - FP: 521.0000 - FN: 521.0000
Metrics: 
 {'loss': 0.6612285375595093, 'ACC': 0.08787878602743149, 'AUC-ROC': 0.8897387385368347, 'AUC-PR': 0.8699610829353333, 'TP': 2779.0, 'TN': 2779.0, 'FP': 521.0, 'FN': 521.0, 'SENSITIVITY': 0.8421212121212122, 'SPECIFICTY': 0.8421212121212122, 'PPV': 0.8421212121212122, 'NPV': 0.8421212121212122, 'F1': 0.8421212121212122}


## End